This script shows all flights currently on route to Kazan airport

In [3]:
import requests
import json
import pandas as pd
import folium
#  ключ лежит в папке выше этого скрипта
import sys, os.path
sys.path.append(os.path.abspath('../'))
from my_keys import avaviationstack_key 
import datetime
print(sys.path)

['c:\\Users\\Mike\\Documents\\Python_Scripts\\Problems_VScode\\UWKD', 'c:\\Users\\Mike\\AppData\\Local\\Programs\\Python\\Python311\\python311.zip', 'c:\\Users\\Mike\\AppData\\Local\\Programs\\Python\\Python311\\DLLs', 'c:\\Users\\Mike\\AppData\\Local\\Programs\\Python\\Python311\\Lib', 'c:\\Users\\Mike\\AppData\\Local\\Programs\\Python\\Python311', '', 'C:\\Users\\Mike\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\Mike\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\Mike\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\Mike\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\Mike\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages', 'c:\\Users\\Mike\\Documents\\Python_Scripts\\Problems_VScode', 'c:\\Users\\Mike\\Documents\\Python_Scripts\\Problems_VScode', 'c:\\Users\\Mike\\Documents\\Python_Scripts\\Problems_VScode']


In [59]:
def load_page(home_icao, offset, limit):
    try:
        api_request = requests.get(f"http://api.aviationstack.com/v1/flights?access_key={avaviationstack_key}&arr_icao={home_icao}&offset={offset}&limit={limit}")
        reply = json.loads(api_request.content)
    except Exception:
        reply = "Error..."
    #  если мы не выбрали все результаты в последнем запросе, делаем следующий с нужным offset
    if reply['pagination']['offset'] + reply['pagination']['count'] < reply['pagination']['total']:
        next_offset = reply['pagination']['offset'] + reply['pagination']['count']
    else:
        next_offset = None
    return reply['data'], next_offset

home_icao = 'UUDD'
offset = 0
result = []

while offset is not None:
    data, offset = load_page(home_icao=home_icao, offset=offset, limit=100)
    result.extend(data)

df = pd.DataFrame.from_dict(result)
df['departure_icao'] = [d.get('icao') for d in df.departure]
print(df.head(5))
# df.to_excel('UWKD flights.xlsx')

  flight_date flight_status  \
0  2023-02-15     scheduled   
1  2023-02-15     scheduled   
2  2023-02-15     scheduled   
3  2023-02-15     scheduled   
4  2023-02-15     scheduled   

                                           departure  \
0  {'airport': None, 'timezone': None, 'iata': 'T...   
1  {'airport': None, 'timezone': None, 'iata': 'O...   
2  {'airport': None, 'timezone': None, 'iata': 'O...   
3  {'airport': None, 'timezone': None, 'iata': 'I...   
4  {'airport': None, 'timezone': None, 'iata': 'B...   

                                             arrival  \
0  {'airport': None, 'timezone': None, 'iata': 'D...   
1  {'airport': None, 'timezone': None, 'iata': 'D...   
2  {'airport': None, 'timezone': None, 'iata': 'D...   
3  {'airport': None, 'timezone': None, 'iata': 'D...   
4  {'airport': None, 'timezone': None, 'iata': 'D...   

                                             airline  \
0  {'name': 'Uzbekistan Airways', 'iata': 'HY', '...   
1  {'name': 'Ural Airlines'

In [60]:
airports = pd.read_csv("airport_code.csv")
print(airports.head(5))

       id ident           type                                name  \
0    6523   00A       heliport                   Total Rf Heliport   
1  323361  00AA  small_airport                Aero B Ranch Airport   
2    6524  00AK  small_airport                        Lowell Field   
3    6525  00AL  small_airport                        Epps Airpark   
4    6526  00AR         closed  Newport Hospital & Clinic Heliport   

   latitude_deg  longitude_deg  elevation_ft continent iso_country iso_region  \
0     40.070801     -74.933601          11.0       NaN          US      US-PA   
1     38.704022    -101.473911        3435.0       NaN          US      US-KS   
2     59.949200    -151.695999         450.0       NaN          US      US-AK   
3     34.864799     -86.770302         820.0       NaN          US      US-AL   
4     35.608700     -91.254898         237.0       NaN          US      US-AR   

   municipality scheduled_service gps_code local_code  
0      Bensalem                no   

In [61]:
# df = pd.read_excel('UWKD flights.xlsx', sheet_name='Sheet1')
# # из файла словарь загржается как стринг
# # чтобы сконвертировать стринг из датафрейма в словарь, ' нужно заменить на ", а None - на "None"
# arrival_string = row['arrival'].replace("'", '"')
# arrival_string = arrival_string.replace('None', '"None"')
# diction = json.loads(arrival_string)
df = pd.merge(df, airports, how='left', left_on=['departure_icao'], right_on=['ident'])

In [62]:
home_coords = airports.loc[airports['ident'] == home_icao, ['latitude_deg', 'longitude_deg']]
print(home_coords)
df_markers = df.loc[df['flight_status'] == 'active', ['municipality', 'latitude_deg', 'longitude_deg', 'arrival']].dropna(axis=0)

       latitude_deg  longitude_deg
53730     55.408798        37.9063


In [63]:
map = folium.Map(location=home_coords, zoom_start=4)
folium.Marker(location=home_coords, popup='Kazan').add_to(map)

for index, row in df_markers.iterrows():
    diction = row['arrival'] # закомментировать, если из файла
    scheduled_time = datetime.datetime.strptime(diction['scheduled'], "%Y-%m-%dT%H:%M:%S%z")
    estimated_time = datetime.datetime.strptime(diction['estimated'], "%Y-%m-%dT%H:%M:%S%z")
    delay = estimated_time - scheduled_time
    if estimated_time > scheduled_time:
        delay_text = f"Arriving with {delay} delay"
    if estimated_time < scheduled_time:
        delay_text = f"Arriving {delay} early"
    else:
        delay_text = "Arriving on time!"

    # из api приходит время по местному времени, несмотря на +00:00.
    # поправку на пояс делать так:
    # tzinfo = datetime.timezone(datetime.timedelta(hours=3, minutes=0))
    # scheduled_time.astimezone(tzinfo).time()

    html = f'''<b>{row['municipality']}</b><br>
                Scheduled arrival: {scheduled_time.time()}<br>
                Estimated arrival: {estimated_time.time()}<br>
                {delay_text}'''
    iframe = folium.IFrame(html,
                        width=250,
                        height=100)
    popup = folium.Popup(iframe, max_width=300)
    marker = folium.Marker((row['latitude_deg'], row['longitude_deg']), popup=popup).add_to(map)

    folium.PolyLine([(row['latitude_deg'], row['longitude_deg']), home_coords], weight=2, color="red").add_to(map)

map
# m.save('line_example.html')